# Table of Contents

1. **Load the Labels**
2. **Important Language Areas**
3. **Read the Data**
4. **Brain Scores**
    - All Voxels
    - Averaged Across ROIs
5. **Forecast Scores for Mean Best Distance**
    - All Voxels
    - Listening vs Reading
    - Only Voxels with Good Brain Scores
    - Averaged Across ROIs


# Load the labels

In [1]:
import matplotlib.pyplot as plt
import os
import mne
import numpy as np
import cortex
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import wilcoxon
import plotly
import plotly.io as pio   
pio.kaleido.scope.mathjax = None
import statsmodels
import os
import matplotlib.pyplot as plt
from voxelwise_tutorials.io import load_hdf5_sparse_array,load_hdf5_array
import os
import numpy as np
import cortex
from cortex.testing_utils import has_installed
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import fdrcorrection
from matplotlib import cm
from matplotlib.colors import ListedColormap
from voxelwise_tutorials.viz import plot_flatmap_from_mapper, _map_to_2d_cmap, plot_2d_flatmap_from_mapper, plot_hist2d, map_voxels_to_flatmap

fsaverage_voxels = 327684
subjects_dir = mne.datasets.sample.data_path() / 'subjects'

In [2]:
import nibabel as nib
import numpy as np

# Load the GIFTI file
gLh = nib.load('mappers/tpl-fsaverage6_hemi-L_desc-MMP_dseg.label.gii')

# Check the data arrays in the file
labels_info=gLh.labeltable.labels
lt_labels = gLh.darrays[0].data

gRh = nib.load('mappers/tpl-fsaverage6_hemi-R_desc-MMP_dseg.label.gii')
rt_labels = gRh.darrays[0].data

In [ ]:
left_data = gLh.agg_data()
right_data = gRh.agg_data()
labels= gLh.labeltable.get_labels_as_dict()
right_data[right_data > 0] += 180
right_labels = gRh.labeltable.get_labels_as_dict()
right_labels = {i + 180: v for i, v in right_labels.items() if i != 0}
labels |= right_labels
label_img = np.concatenate((left_data, right_data))

labels = {k: v.removesuffix("_ROI") for k, v in labels.items()}
labels


In [ ]:
data_fs7 = cortex.freesurfer.upsample_to_fsaverage(label_img, "fsaverage6", subjects_dir)
data_fs7.shape

important language areas

In [5]:
early_visual_rois = ['V1','V2']
early_indices = [1, 4]

vwfa_indices = [  138, 136]
vwfa_rois = ['PH','TE2P']

early_auditory_rois = ['LBelt','PBelt','MBelt','RI','A4']
early_auditory_indices = [24, 174, 124, 173, 104, 175]

primary_auditory_rois = ['A1', 'STV']
primary_auditory_indices = [24, 28]

secondary_auditory_rois = ['STV']
secondary_auditory_indices = [28]


late_language_rois = ['A5','44','45', 'IFJa', 'IFSp', 'PGi', 'PGp', 'PGs', 'TPOJ1', 'TPOJ2', 'TPOJ3', 'STGa','STSda', 'STSdp','TA2', 'STSva', 'STSvp', 'PFm']
late_language_indices = [125, 74, 75, 79, 81, 150, 143, 151, 139, 140, 141, 123, 128, 129, 107, 176, 130, 149]

main_language_rois = ['EV','VWFA','EAC','LL','A1']
main_language_indices = [early_indices, vwfa_indices, early_auditory_indices, late_language_indices, primary_auditory_indices]

angular_gyrus_rois = ['PFm','PGs', 'PGi', 'TPOJ2','TPOJ3', 'PGp']
angular_gyrus_indices = [149, 151, 150, 140, 141, 143]

ltc_rois = ['STSda', 'STSva', 'STGa', 'TE1a', 'TE2a', 'TGv', 'TGd', 'A5', 'STSdp', 'STSvp', 'PSL', 'STV', 'TPOJ1']
ltc_indices = [128, 176 ,123, 132, 134, 172, 131, 125, 129, 130, 25, 28, 139]

ifg_rois = ['44', '45', 'IFJa', 'IFSp']
ifg_indies = [74, 75, 79, 81]

mfg_rois = ['55b']
mfg_indices = [12]

ifgorb_rois = ['47m','47l', '47s' ]
ifgorb_indices = [66, 76, 94]

late_language_main_rois = ['AG', 'LTC', 'IFG', 'MFG', 'IFGOrb']
late_language_main_indices = [angular_gyrus_indices, ltc_indices, ifg_indies, mfg_indices, ifgorb_indices]

broca_rois = ['44','45']
broca_indices = [74, 75]

semantic_rois = ['IFJa', 'IFJp', 'PGi', 'PGp', 'PGs', 'TPOJ1', 'TPOJ2', 'TPOJ3']
semantic_indices = [79,80, 150, 143, 151, 139, 140, 141]

phonology_rois = ['STSda', 'STSva', 'STSdp', 'STSvp', 'STGa']
phonology_indices = [128, 176,  129, 130, 123 ]

lang_audition_rois = ['TA2']
lang_audition_indices= [107]

late_language_func_rois = ['Broca\'s area (production and grammar)', 'IFJ & PG & TPOJ (semantic)', 'STGa & STS (auditory linguistic integration)', 'TA2 (language auditory processing)' ]
late_language_func_indices = [broca_indices, semantic_indices, phonology_indices, lang_audition_indices]

stsa_rois = ['STSda', 'STSva']
stsa_indices = [128, 176]

stsp_rois = ['STSdp', 'STSvp']
stsp_indices = [129, 130]

pars_opercularis_rois = ['44']
pars_opercularis_indices = [74]

pars_triangularis_rois = ['45']
pars_triangularis_indices = [75]

language_paper_rois = ['STSa', 'STSp', 'pars obercularis', 'pars triangularis', 'AG']
language_paper_indices = [stsa_indices, stsp_indices, pars_opercularis_indices,  pars_triangularis_indices, angular_gyrus_indices]

#Broca's area is vital for speech production and grammar comprehension
#IFJ, PG, TPOJ involved in semantic processing
#STSGa, STS play roles in phonological processing and auditory-linguistic integration
#TA2 is implicated in auditory processing especially in the context of language

# Read the data

In [ ]:
from voxelwise_tutorials.io import load_hdf5_sparse_array
import os
import numpy as np
import cortex

modality = 'listening'
layer = 8
directory = 'mappers/'
subjects = ['0'+str(i) for i in range(1,10) if i not in [4,6]]
distances = np.arange(-5, 30)
n_distances = len(distances)
n_subjects = len(subjects)

listening_ceilings = np.zeros((n_subjects, fsaverage_voxels))
listening_brain_scores = np.zeros((n_subjects, fsaverage_voxels))
listening_forecast_brain_scores = np.zeros((n_subjects, n_distances, fsaverage_voxels))
listening_forecast_scores = np.zeros((n_subjects, n_distances, fsaverage_voxels))
listening_precentage_increase = np.zeros((n_subjects, n_distances, fsaverage_voxels))
listening_ceiled_forecast_brain_scores =  np.zeros((n_subjects, n_distances, fsaverage_voxels))

for i in range(len(subjects)):
    eachsub = subjects[i] 
    mapper_file = os.path.join(directory, f'subject{eachsub}_mappers.hdf')
    voxel_to_fsaverage = load_hdf5_sparse_array(mapper_file, key='voxel_to_fsaverage')
    ceiling = np.load('noise_ceiling/results/'+modality+'/subject_'+str(eachsub)+'_kernel_ridge.npy')
    ceiling_voxcorrs = voxel_to_fsaverage @ ceiling
    listening_ceilings[i] = ceiling_voxcorrs
    model_voxcorrs = np.load('predictions2/'+modality+'/'+eachsub+'/'+str(layer)+'/layer_'+str(layer)+'.npy')
    listening_brain_scores[i] = voxel_to_fsaverage @ model_voxcorrs
    for d in distances:
        forecast_model_voxcorrs = np.load('predictions2/future_'+modality+'/'+str(eachsub)+'/'+str(layer)+'/'+str(d)+'/layer_'+str(layer)+'.npy')
        listening_forecast_brain_scores[i,d+5,:] = voxel_to_fsaverage @ forecast_model_voxcorrs
        listening_ceiled_forecast_brain_scores[i, d+5,:] = listening_forecast_brain_scores[i,d+5,:] / ceiling_voxcorrs
        listening_forecast_scores[i, d+5,:] = listening_forecast_brain_scores[i,d+5,:] - listening_brain_scores[i]
        listening_precentage_increase[i,d+5,:] = (listening_forecast_brain_scores[i,d+5,:] / listening_brain_scores[i]) - 1
mean_listening_ceiling = np.nanmean(listening_ceilings, axis=0)
filtered_listening_forecast_scores = listening_forecast_scores * (mean_listening_ceiling>=0.05)
filtered_listening_forecast_scores[filtered_listening_forecast_scores==0] = None


In [ ]:
from voxelwise_tutorials.io import load_hdf5_sparse_array
import os
import numpy as np
import cortex

modality = 'reading'
layer = 8
directory = 'mappers/'
subjects = ['0'+str(i) for i in range(1,10) if i not in [4,6]]
distances = np.arange(-5, 30)
n_distances = len(distances)
n_subjects = len(subjects)

reading_ceilings = np.zeros((n_subjects, fsaverage_voxels))
reading_brain_scores = np.zeros((n_subjects, fsaverage_voxels))
reading_forecast_brain_scores = np.zeros((n_subjects, n_distances, fsaverage_voxels))
reading_forecast_scores = np.zeros((n_subjects, n_distances, fsaverage_voxels))
reading_precentage_increase = np.zeros((n_subjects, n_distances, fsaverage_voxels))
reading_ceiled_forecast_brain_scores =  np.zeros((n_subjects, n_distances, fsaverage_voxels))

for i in range(len(subjects)):
    eachsub = subjects[i] 
    mapper_file = os.path.join(directory, f'subject{eachsub}_mappers.hdf')
    voxel_to_fsaverage = load_hdf5_sparse_array(mapper_file, key='voxel_to_fsaverage')
    ceiling = np.load('noise_ceiling/results/'+modality+'/subject_'+str(eachsub)+'_kernel_ridge.npy')
    ceiling_voxcorrs = voxel_to_fsaverage @ ceiling
    reading_ceilings[i] = ceiling_voxcorrs
    model_voxcorrs = np.load('predictions2/'+modality+'/'+eachsub+'/'+str(layer)+'/layer_'+str(layer)+'.npy')
    reading_brain_scores[i] = voxel_to_fsaverage @ model_voxcorrs
    for d in distances:
        forecast_model_voxcorrs = np.load('predictions2/future_'+modality+'/'+str(eachsub)+'/'+str(layer)+'/'+str(d)+'/layer_'+str(layer)+'.npy')
        reading_forecast_brain_scores[i,d+5,:] = voxel_to_fsaverage @ forecast_model_voxcorrs
        reading_ceiled_forecast_brain_scores[i, d+5,:] = reading_forecast_brain_scores[i,d+5,:] / ceiling_voxcorrs
        reading_forecast_scores[i, d+5,:] = reading_forecast_brain_scores[i,d+5,:] - reading_brain_scores[i]
        reading_precentage_increase[i,d+5,:] = (reading_forecast_brain_scores[i,d+5,:] / reading_brain_scores[i]) - 1
mean_reading_ceiling = np.nanmean(reading_ceilings, axis=0)
filtered_reading_forecast_scores = reading_forecast_scores * (mean_reading_ceiling>=0.05)
filtered_reading_forecast_scores[filtered_reading_forecast_scores==0] = None


# Brain scores

In [ ]:
listening_brain_scores.mean(), reading_brain_scores.mean()

## All voxels

Listening

In [ ]:
vertex_list_BS = cortex.Vertex(listening_brain_scores.mean(0),'fsaverage', vmin=0.05, vmax=0.2, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_BS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_BS)

Reading

In [ ]:
vertex_read_BS = cortex.Vertex(reading_brain_scores.mean(0),'fsaverage', vmin=0.05, vmax=0.2, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_BS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_BS)

## Averaged across ROIs

### Listening

In [ ]:
import numpy as np

scores = listening_brain_scores.mean(0)

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
brain_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_list_BS_ROIs = cortex.Vertex(brain_scores_across_ROIs,'fsaverage', vmin=0.05, vmax=0.2, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_BS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_BS_ROIs)

### Reading

In [ ]:
import numpy as np

scores = reading_brain_scores.mean(0)

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
brain_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_read_BS_ROIs = cortex.Vertex(brain_scores_across_ROIs, 'fsaverage', vmin=0.05, vmax=0.2, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_BS_ROIs, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_BS_ROIs)

# Forecast scores for mean best distance

In [ ]:
# Best mean distance for listening
best_listening_mean_distance = np.argmax(listening_forecast_scores.mean((0,2)), axis=0) - 5
best_reading_mean_distance = np.argmax(reading_forecast_scores.mean((0,2)), axis=0) - 5
best_listening_mean_distance, best_reading_mean_distance

## All Voxels

Listening

In [ ]:
mean_listening_forecast_scores_at_best_dist = listening_forecast_scores[:,best_listening_mean_distance+5,:].mean(0)
vertex_list_FS = cortex.Vertex(mean_listening_forecast_scores_at_best_dist,'fsaverage', vmin=0.005, vmax=0.02, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_FS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_FS)

Reading

In [ ]:
mean_reading_forecast_scores_at_best_dist = reading_forecast_scores[:,best_reading_mean_distance+5,:].mean(0)

vertex_read_FS = cortex.Vertex(mean_reading_forecast_scores_at_best_dist,'fsaverage', vmin=0.005, vmax=0.02, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_FS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_FS)

listening vs reading

In [ ]:
vertex_list_read_FS = cortex.Vertex2D(mean_listening_forecast_scores_at_best_dist,mean_reading_forecast_scores_at_best_dist,'fsaverage', vmin=0, vmax=0.01, vmin2=0, vmax2=0.01)

fig = cortex.quickshow(vertex_list_read_FS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_FS)

## Only voxels with good brain score

listening

In [ ]:
filtered_mean_listening_forecast_scores_at_best_dist = mean_listening_forecast_scores_at_best_dist * (listening_forecast_brain_scores[:,best_listening_mean_distance+5,:].mean(0)> 0.05)
vertex_list_FS_filtered = cortex.Vertex(filtered_mean_listening_forecast_scores_at_best_dist,'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_FS_filtered, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_FS_filtered)

Reading

In [ ]:
filtered_mean_reading_forecast_scores_at_best_dist = mean_reading_forecast_scores_at_best_dist * (reading_forecast_brain_scores[:,best_reading_mean_distance+5,:].mean(0) > 0.05)
vertex_read_FS_filtered = cortex.Vertex(filtered_mean_reading_forecast_scores_at_best_dist, 'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_FS_filtered, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_FS_filtered)

Listening vs reading

In [ ]:
vertex_list_read_FS_filtered = cortex.Vertex2D(filtered_mean_listening_forecast_scores_at_best_dist, filtered_mean_reading_forecast_scores_at_best_dist, 'fsaverage', vmin=0, vmax=0.01, vmin2=0, vmax2=0.01)

fig = cortex.quickshow(vertex_list_read_FS_filtered, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_FS_filtered)

## Averaged across ROIs

listening

In [ ]:
import numpy as np

scores = mean_listening_forecast_scores_at_best_dist

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
list_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_list_FS_ROIs = cortex.Vertex(list_scores_across_ROIs,'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_BS_ROIs)

Reading

In [ ]:
import numpy as np

scores = mean_reading_forecast_scores_at_best_dist

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
read_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_read_FS_ROIs = cortex.Vertex(read_scores_across_ROIs, 'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_read_FS_ROIs)

Listening vs reading

In [ ]:
vertex_list_read_FS = cortex.Vertex2D(list_scores_across_ROIs,read_scores_across_ROIs,'fsaverage', vmin=0, vmax=0.01, vmin2=0, vmax2=0.01)

fig = cortex.quickshow(vertex_list_read_FS, colorbar_location='right',
                       with_rois=True)
fig.show()
cortex.webshow(vertex_list_read_FS)

## Only voxels with good brain scores averaged across ROIS

listening

In [ ]:
import numpy as np

scores = filtered_mean_listening_forecast_scores_at_best_dist

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
list_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_list_FS_ROIs = cortex.Vertex(list_scores_across_ROIs,'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_BS_ROIs)

reading

In [ ]:
import numpy as np

scores = filtered_mean_reading_forecast_scores_at_best_dist

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
read_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_read_FS_ROIs = cortex.Vertex(read_scores_across_ROIs, 'fsaverage', vmin=0, vmax=0.01, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_read_FS_ROIs)

listening vs reading

In [ ]:
vertex_list_read_FS = cortex.Vertex2D(list_scores_across_ROIs,read_scores_across_ROIs,'fsaverage', vmin=0, vmax=0.01, vmin2=0, vmax2=0.01)

fig = cortex.quickshow(vertex_list_read_FS, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_FS)

# Best Distance

## All Voxels

listening

In [ ]:
listening_forecast_brain_scores.argmax(1).mean(0) - 5

In [ ]:
best_listening_distances = listening_forecast_brain_scores.mean(0).argmax(0) - 5
vertex_list_dist = cortex.Vertex(best_listening_distances,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_dist, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_dist)

reading

In [ ]:
best_reading_distances = reading_forecast_brain_scores.mean(0).argmax(0) - 5
vertex_read_dist = cortex.Vertex(best_reading_distances, 'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_dist, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_dist)

listening vs reading

In [ ]:
vertex_list_read_dist = cortex.Vertex2D(best_listening_distances,best_reading_distances,'fsaverage', vmin=0, vmax=15, vmin2=0, vmax2=15)

fig = cortex.quickshow(vertex_list_read_dist, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_dist)

## Only voxels with good brain scores

listening

In [ ]:
# Filtering based on good forecast score
filt_best_listening_distances = best_listening_distances * (listening_forecast_brain_scores.mean(0).max(0) > 0.05)
vertex_list_dist_filt = cortex.Vertex(filt_best_listening_distances,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_dist_filt, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_dist_filt)

In [ ]:
# Filtering based on good brain scores
filt_best_listening_distances2 = best_listening_distances * (listening_brain_scores.mean(0) > 0.05)
vertex_list_dist_filt2 = cortex.Vertex(filt_best_listening_distances2,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_dist_filt2, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_dist_filt2)

In [ ]:
# Voxels that has forecast brain score >0.05 but brain score less
vertex_list_dist_diff = cortex.Vertex(filt_best_listening_distances - filt_best_listening_distances2,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_dist_diff, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_dist_diff)

reading

In [ ]:
# Filtering based on the best forecast score
filt_best_reading_distances = best_reading_distances * (reading_forecast_brain_scores.mean(0).max(0) > 0.05)
vertex_read_dist_filt = cortex.Vertex(filt_best_reading_distances,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_dist_filt, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_dist_filt)

In [ ]:
# Filtering based on the brain scores
filt_best_reading_distances2 = best_reading_distances * (reading_brain_scores.mean(0) > 0.05)
vertex_read_dist_filt2 = cortex.Vertex(filt_best_reading_distances2,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_dist_filt2, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_dist_filt2)

In [ ]:
# Voxels that has forecast brain score >0.05 but brain score less
vertex_read_best_dist = cortex.Vertex(filt_best_reading_distances - filt_best_reading_distances2,'fsaverage', vmin=0, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_best_dist, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_read_best_dist)

listening vs reading

## Averaged across ROIs

listening

In [ ]:
# Compute the average best dist per label
unique_labels = np.unique(data_fs7)
label_to_avg_dist = {label: np.mean(best_listening_distances[data_fs7 == label]) for label in unique_labels}

# Map averages back to original voxel positions
list_dist_across_ROIs = np.array([label_to_avg_dist[label] for label in data_fs7])

vertex_list_dist_ROIs = cortex.Vertex(list_dist_across_ROIs,'fsaverage', vmin=2, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_dist_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_dist_ROIs)

In [ ]:
import numpy as np


# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg= {label: listening_forecast_scores[:,round(label_to_avg_dist[label]),data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
list_max_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_list_max_FS_ROIs = cortex.Vertex(list_max_scores_across_ROIs,'fsaverage', vmin=0.0005, vmax=0.006, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_max_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_max_FS_ROIs)

In [ ]:
## In this cell I show the maximum forecast score for each voxel, averaged across ROIS
# This at ;east show that for each voxel it comes a time where the forecast score is high
# Not necessarily in the same best distance for the whole ROI
scores = listening_forecast_scores.mean(0).max(0)

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
list_max_scores_across_ROIs2 = np.array([label_to_avg[label] for label in data_fs7])

vertex_list_max_FS_ROIs2 = cortex.Vertex(list_max_scores_across_ROIs2,'fsaverage', vmin=0.01, vmax=0.03, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_list_max_FS_ROIs2, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_list_max_FS_ROIs2)

reading

In [ ]:
# Compute the average best dist per label
unique_labels = np.unique(data_fs7)
label_to_avg_dist = {label: np.mean(best_reading_distances[data_fs7 == label]) for label in unique_labels}

# Map averages back to original voxel positions
read_dist_across_ROIs = np.array([label_to_avg_dist[label] for label in data_fs7])

vertex_read_dist_ROIs = cortex.Vertex(read_dist_across_ROIs,'fsaverage', vmin=2, vmax=15, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_dist_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_read_dist_ROIs)

In [ ]:
# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg= {label: reading_forecast_scores[:,round(label_to_avg_dist[label]),data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
read_max_scores_across_ROIs = np.array([label_to_avg[label] for label in data_fs7])

vertex_read_max_FS_ROIs = cortex.Vertex(read_max_scores_across_ROIs,'fsaverage', vmin=0.0005, vmax=0.006, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_max_FS_ROIs, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_read_max_FS_ROIs)

In [ ]:
## In this cell I show the maximum forecast score for each voxel, averaged across ROIS
# This at ;east show that for each voxel it comes a time where the forecast score is high
# Not necessarily in the same best distance for the whole ROI
scores = reading_forecast_scores.mean(0).max(0)

# Compute the average score per label
unique_labels = np.unique(data_fs7)
label_to_avg = {label: scores[data_fs7 == label].mean() for label in unique_labels}

# Map averages back to original voxel positions
read_max_scores_across_ROIs2 = np.array([label_to_avg[label] for label in data_fs7])

vertex_read_max_FS_ROIs2 = cortex.Vertex(read_max_scores_across_ROIs2,'fsaverage', vmin=0.01, vmax=0.03, cmap='YlGnBu_r')

fig = cortex.quickshow(vertex_read_max_FS_ROIs2, colorbar_location='right',
                    with_rois=True)
fig.show()
#cortex.webshow(vertex_read_max_FS_ROIs2)

listening vs reading

In [ ]:
vertex_list_read_dist_ROI = cortex.Vertex2D(list_dist_across_ROIs,read_dist_across_ROIs,'fsaverage', vmin=0, vmax=15, vmin2=0, vmax2=15)

fig = cortex.quickshow(vertex_list_read_dist_ROI, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_dist_ROI)

In [ ]:
vertex_list_read_max_score_ROI = cortex.Vertex2D(list_max_scores_across_ROIs,read_max_scores_across_ROIs,'fsaverage', vmin=0.0005, vmax=0.01, vmin2=0.0005, vmax2=0.01)

fig = cortex.quickshow(vertex_list_read_max_score_ROI, colorbar_location='right',
                       with_rois=True)
fig.show()
#cortex.webshow(vertex_list_read_max_score_ROI)